In [11]:
#importing libraries
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import os
from sklearn.cluster import KMeans
 
from geopy.geocoders import Nominatim 

import matplotlib.cm as cm
import matplotlib.colors as colors

In [12]:
!pip install geocoder
import geocoder

In [8]:
!pip install folium
import folium

     |████████████████████████████████| 94 kB 6.4 MB/s  eta 0:00:01


In [3]:
# Send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India").text
# Parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
# Create a list to store neighbourhood data
neighborhoodList = []
# Append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
  neighborhoodList.append(row.text)
# Create a new DataFrame from the list
hyd_df = pd.DataFrame({"Neighborhood": neighborhoodList})
hyd_df.head()

,Neighborhood
0,A. C. Guards
1,A. S. Rao Nagar
2,Abhyudaya Nagar
3,Abids
4,Adibatla


In [4]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hyderabad, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:

# Call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in hyd_df["Neighborhood"].tolist()]


Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)


In [10]:
# Create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# Merge the coordinates into the original dataframe
hyd_df['Latitude'] = df_coords['Latitude']
hyd_df['Longitude'] = df_coords['Longitude']
print(hyd_df.shape)
hyd_df

(200, 3)


,Neighborhood,Latitude,Longitude
0,A. C. Guards,17.395015,78.459812
1,A. S. Rao Nagar,17.411200,78.508240
2,Abhyudaya Nagar,17.337650,78.564140
3,Abids,17.389800,78.476580
4,Adibatla,17.235790,78.541300
...,...,...,...
195,Secunderabad,17.442000,78.501920
196,Serilingampally,17.482160,78.323000
197,Shah-Ali-Banda,17.357390,78.473200
198,Shahran Market,17.364890,78.476290


In [13]:
# Getting the coordinates of Hyderabad
address = 'Hyderabad, India'
geolocator = Nominatim(user_agent="hyd_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hyderabad, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hyderabad, India 17.38878595, 78.46106473453146.


In [14]:
map_hyd = folium.Map(location=[latitude, longitude], zoom_start=11)
# Adding markers to map
for lat, lng, neighborhood in zip(hyd_df['Latitude'],  hyd_df['Longitude'], hyd_df['Neighborhood']):
 label = '{}'.format(neighborhood)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_hyd)
map_hyd

In [15]:
CLIENT_ID = 'R3RXMPUY3TFHL1ZNYGVAGXK53E0EVQ5E5WHNUUJT4AYCWV3A' #  Foursquare ID
CLIENT_SECRET = '5Z11YMPXKUUXZEPOH5TYCV4SEYWQSDKTAZ0XNVRPELOUQAHJ' #  Foursquare Secret
VERSION = '20180604'
def getbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [17]:
hyd_venues = getbyVenues(names=hyd_df['Neighborhood'],
                                   latitudes=hyd_df['Latitude'],
                                   longitudes=hyd_df['Longitude']
                                  )

A. C. Guards
A. S. Rao Nagar
Abhyudaya Nagar
Abids
Adibatla
Adikmet
Afzal Gunj
Aghapura
Aliabad, Hyderabad
Alijah Kotla
Allwyn Colony
Alwal
Amberpet
Ameenpur
Ameerpet
Anandbagh
Ashok Nagar, Hyderabad
Asif Nagar
Attapur
Azamabad, Hyderabad
Azampura
Badichowdi
Bagh Lingampally
Bairamalguda
Balapur, Ranga Reddy district
Balkampet
Banjara Hills
Bank Street, Hyderabad
Barkas, Hyderabad
Barkatpura
Basheerbagh
Bazarghat
Begum Bazaar
Begumpet
Bharat Nagar
BHEL Township, Hyderabad
Boggulkunta
Bolarum
Borabanda
Bowenpally
Chaderghat
Champapet
Chanchalguda
Chandrayan Gutta
Chatta Bazaar
Chengicherla
Cherlapally
Chikkadpally
Chilkalguda
Chintal Basti
Chintalakunta
Dabirpura
Dar-ul-Shifa
Dhoolpet
Dilsukhnagar
Domalguda
Dundigal
ECIL X Roads
Edi Bazar
Erragadda
Fateh Nagar, Hyderabad
Ferozguda
Film Nagar
Gachibowli
Gaddiannaram
Gautham Nagar
Golnaka
Goshamahal
Gudimalkapur
Gulzar Houz
Gundlapochampalli
Habsiguda
Hafeezpet
Hasmathpet
Hastinapuram
Hayathnagar
Himayatnagar, Hyderabad
HITEC City
Hydergu

In [18]:
hyd_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,A. C. Guards,17.395015,78.459812,Hotel Marjaan,17.395373,78.455115,Café
1,A. C. Guards,17.395015,78.459812,Milan Juice Center,17.392266,78.458102,Snack Place
2,A. S. Rao Nagar,17.411200,78.508240,Srinivasa Textiles,17.407338,78.509908,Clothing Store
3,A. S. Rao Nagar,17.411200,78.508240,Reign Gym & Spa,17.407340,78.509920,Gym
4,A. S. Rao Nagar,17.411200,78.508240,turnhalle gym,17.407347,78.509941,Gym


In [19]:
hyd_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
A. C. Guards,2,2,2,2,2,2
A. S. Rao Nagar,4,4,4,4,4,4
Abhyudaya Nagar,6,6,6,6,6,6
Abids,19,19,19,19,19,19
Adikmet,3,3,3,3,3,3
...,...,...,...,...,...,...
Saroornagar,6,6,6,6,6,6
Secunderabad,11,11,11,11,11,11
Shah-Ali-Banda,6,6,6,6,6,6


In [33]:
hyd_venues['Venue Category'].unique()

array(['Café', 'Snack Place', 'Clothing Store', 'Gym', 'Ice Cream Shop',
       'Indian Restaurant', 'Hotel', 'English Restaurant', 'Burger Joint',
       "Women's Store", 'Diner', 'Juice Bar', 'Shoe Store',
       'Neighborhood', 'Department Store', 'Mobile Phone Shop',
       'Shopping Mall', 'Fast Food Restaurant', 'Pizza Place', 'Platform',
       'Breakfast Spot', 'Bakery', 'Afghan Restaurant', 'Dessert Shop',
       'Middle Eastern Restaurant', 'Hotel Bar', 'ATM', 'Bar',
       'Vegetarian / Vegan Restaurant', 'Multiplex', 'Tea Room', 'Market',
       'Movie Theater', 'Brewery', 'Chinese Restaurant', 'Bus Station',
       'Train Station', 'Indie Movie Theater', 'Coffee Shop', 'Arcade',
       'Night Market', 'Garden Center', 'Drive-in Theater',
       'Hockey Arena', 'Light Rail Station', 'Sandwich Place',
       'South Indian Restaurant', 'Falafel Restaurant', 'Bank',
       'Smoke Shop', 'Restaurant', 'Gaming Cafe', 'Hookah Bar',
       'Gym / Fitness Center', 'Gift Shop', 'Jew

In [21]:
# One hot encoding
hyd_onehot = pd.get_dummies(hyd_venues[['Venue Category']], prefix="", prefix_sep="")
# Adding neighborhood column back to dataframe
hyd_onehot['Neighborhoods'] = hyd_venues['Neighborhood']
# Moving neighbourhood column to the first column
fixed_columns = [hyd_onehot.columns[-1]] + list(hyd_onehot.columns[:-1])
hyd_onehot = hyd_onehot[fixed_columns]
print(hyd_onehot.shape)


(961, 147)


In [22]:
hyd_grouped=hyd_onehot.groupby(["Neighborhoods"]).sum().reset_index()
print(hyd_grouped.shape)
hyd_grouped

(171, 147)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Asian Restaurant,Auto Garage,BBQ Joint,...,Tea Room,Tennis Stadium,Tex-Mex Restaurant,Theme Park,Trail,Train Station,Vegetarian / Vegan Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,A. C. Guards,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abhyudaya Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Abids,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,Adikmet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,Saroornagar,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
167,Secunderabad,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
168,Shah-Ali-Banda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
169,Shahran Market,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
#Clustering and finding the Pizza Place in the neighborhood
len((hyd_grouped[hyd_grouped["Pizza Place"] > 0]))

21

In [35]:
# Creating a dataframe for Pizza Place data only
hyd_house = hyd_grouped[["Neighborhoods","Pizza Place"]]

In [36]:
# Setting the number of clusters
kclusters = 3
hyd_clustering = hyd_house.drop(["Neighborhoods"], 1)
# Run k-means clustering algorithm
kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(hyd_clustering)
# Checking cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0], dtype=int32)

In [37]:
# Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
hyd_merged = hyd_house.copy()
# Add the clustering labels
hyd_merged["Cluster Labels"] = kmeans.labels_
hyd_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
hyd_merged.head(10)

,Neighborhood,Pizza Place,Cluster Labels
0,A. C. Guards,0,0
1,A. S. Rao Nagar,0,0
2,Abhyudaya Nagar,0,0
3,Abids,0,0
4,Adikmet,1,1
5,Afzal Gunj,1,1
6,Aghapura,0,0
7,"Aliabad, Hyderabad",0,0
8,Alijah Kotla,0,0
9,Allwyn Colony,0,0


In [38]:
 # Adding latitude and longitude values to the existing dataframe
hyd_merged['Latitude'] = hyd_df['Latitude']
hyd_merged['Longitude'] = hyd_df['Longitude']
# Sorting the results by Cluster Labels
hyd_merged.sort_values(["Cluster Labels"], inplace=True)
hyd_merged

,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
0,A. C. Guards,0,0,17.395015,78.459812
104,Mahatma Gandhi Road (Secunderabad),0,0,17.487350,78.420870
105,Malakpet,0,0,17.481130,78.583700
106,Malkajgiri,0,0,17.512650,78.441290
107,Malkajgiri mandal,0,0,17.361450,78.472090
...,...,...,...,...,...
114,Mehdipatnam,1,1,17.350670,78.534040
40,Chilkalguda,1,1,17.377330,78.489740
66,Izzat Nagar,1,1,17.391920,78.505640
145,Punjagutta,1,1,17.433510,78.566730


In [39]:
# Creating the map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# Setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hyd_merged['Latitude'], hyd_merged['Longitude'], hyd_merged['Neighborhood'], hyd_merged['Cluster Labels']):
 label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
 folium.CircleMarker([lat,lon],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [40]:
len(hyd_merged.loc[hyd_merged['Cluster Labels'] == 0])


150

In [41]:
len(hyd_merged.loc[hyd_merged['Cluster Labels'] == 1])


20

In [42]:
len(hyd_merged.loc[hyd_merged['Cluster Labels'] == 2])

1